# 基于mindspore实现Q-Learning的示例
   
   本小节介绍基于mindspore实现Q-Learning的示例，参考作者mofan用到的一个名为“treasure on right”的简单游戏来实现Q-Learning这个算法


案例介绍：宝藏treasure在最右边的位置。然后训练一个智能体Agent去获得这个宝藏


### 参数解释：

  N_STATES ：表示状态数量。其实就是位置数量

  EPSILON： ϵ-greedy 的ϵ  

同理，ALPHA，GAMMA也类似

  MAX_EPISODES ： 表示玩多少轮游戏来训练

  FRESH_TIME：用来输出的参数，控制多久刷新一次页面之类

  TerminalFlag： 用来记录游戏结束的标志符，方便统一，就放在外面

### 导入库和函数

In [1]:
import time
import numpy as np
import pandas as pd

### 定义环境和状态并定义每个动作的奖励R和状态转移S

In [7]:
N_STATES = 6                      #定义的环境下，有6个状态
ACTIONS = ["left", "right"]       #有两个动作
EPSILON = 0.9
ALPHA = 0.1
GAMMA = 0.9
MAX_EPISODES = 15
FRESH_TIME = 0.3
TerminalFlag = "terminal"         #有一个终止状态
def build_q_table(n_states, actions):   # build_q_table 函数用于建立Q表，初始化为全0的dataframe
    return pd.DataFrame(
        np.zeros((n_states, len(actions))),
        columns=actions
    )
def choose_action(state, q_table):  # choose_action函数用于根据当前状态和Q表选择动作，这里采用了epsilon-greedy的策略，以一定的概率随机选择动作或选择Q值最大的动作
    state_table = q_table.loc[state, :]
    if (np.random.uniform() > EPSILON) or ((state_table == 0).all()):
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_table.idxmax()
    return action_name
def get_env_feedback(S, A):       # get_env_feedback 函数用于得到当前状态和采取动作后的下一个状态和奖励值。
    if A == "right":
        if S == N_STATES - 2:
            S_, R = TerminalFlag, 1
        else:
            S_, R = S + 1, 0
    else:
        S_, R = max(0, S - 1), 0
    return S_, R

def update_env(S, episode, step_counter):  # update_env(S, episode, step_counter)函数用于更新环境，展示当前状态和可选的动作
    env_list = ["-"] * (N_STATES - 1) + ["T"]
    if S == TerminalFlag:
        interaction = 'Episode %s: total_steps = %s' % (episode + 1, step_counter)
        print(interaction)
        time.sleep(2)
    else:
        env_list[S] = '0'
        interaction = ''.join(env_list)
        print(interaction)
        time.sleep(FRESH_TIME)

def rl():             #rl()函数用于训练智能体，训练过程中根据当前状态和Q表选择动作，执行动作并更新Q表
    q_table = build_q_table(N_STATES, ACTIONS)
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False
        update_env(S, episode, step_counter)
        while not is_terminated:
            A = choose_action(S, q_table)
            S_, R = get_env_feedback(S, A)
            q_predict = q_table.loc[S, A]

            if S_ != TerminalFlag:
                q_target = R + GAMMA * q_table.loc[S_, :].max()
            else:
                q_target = R
                is_terminated = True
            q_table.loc[S, A] += ALPHA * (q_target - q_predict)
            S = S_
            update_env(S, episode, step_counter + 1)
            step_counter += 1
    return q_table


### 训练结果Q表

In [8]:
if __name__ == '__main__':
    q_table = rl()
    print(q_table)

0----T
0----T
-0---T
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 1: total_steps = 9
0----T
-0---T
--0--T
---0-T
----0T
Episode 2: total_steps = 5
0----T
-0---T
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 3: total_steps = 8
0----T
0----T
-0---T
0----T
0----T
-0---T
--0--T
---0-T
----0T
Episode 4: total_steps = 9
0----T
-0---T
--0--T
---0-T
----0T
Episode 5: total_steps = 5
0----T
-0---T
--0--T
---0-T
----0T
Episode 6: total_steps = 5
0----T
-0---T
--0--T
---0-T
----0T
Episode 7: total_steps = 5
0----T
-0---T
--0--T
---0-T
----0T
Episode 8: total_steps = 5
0----T
-0---T
--0--T
---0-T
--0--T
---0-T
----0T
Episode 9: total_steps = 7
0----T
-0---T
--0--T
---0-T
----0T
Episode 10: total_steps = 5
0----T
-0---T
--0--T
---0-T
----0T
Episode 11: total_steps = 5
0----T
-0---T
--0--T
---0-T
----0T
---0-T
----0T
Episode 12: total_steps = 7
0----T
-0---T
--0--T
---0-T
----0T
Episode 13: total_steps = 5
0----T
-0---T
--0--T
---0-T
----0T
Episode 14: total_steps = 5
0----T
-0---T
--0--T
-